In [1]:
import pandas as pd
import numpy as np
import os

In [11]:
import os

def count_lines(file_path):
    """Returns the number of lines in a file."""
    with open(file_path, 'r') as f:
        return sum(1 for _ in f)

def check_files(original_dir='/dsi/scratch/home/dsi/orrbavly/corona_data/original_files/', embeddings_dir='/dsi/scratch/home/dsi/orrbavly/corona_data/embeddings/'):
    mismatched_files = []
    counter = 0
    for file_name in os.listdir(original_dir):
        original_path = os.path.join(original_dir, file_name)
        base_name, ext = os.path.splitext(file_name)
        embedding_file_name = f"{base_name}_embedded{ext}"
        embedding_path = os.path.join(embeddings_dir, embedding_file_name)
        
        if os.path.isfile(original_path) and os.path.isfile(embedding_path):
            print(f"Working on file: {file_name}")
            counter += 1
            original_lines = count_lines(original_path)
            embedding_lines = count_lines(embedding_path)
            
            if original_lines != embedding_lines:
                mismatched_files.append(file_name)
    print(f"Counter: {counter}")
    for file in mismatched_files:
        print(file)
    print(f"Total mismatched files: {len(mismatched_files)}")

if __name__ == "__main__":
    check_files()


Working on file: 03855000011616_TCRB.csv
Working on file: 03855000011623_TCRB.csv
Working on file: 03855000011624_TCRB.csv
Working on file: 03855000011628_TCRB.csv
Working on file: 03855000011629_TCRB.csv


KeyboardInterrupt: 

In [5]:
import os
import shutil
import sys

def move_files(file_list, source_dir, destination_dir):
      # Ensure source directory exists
    if not os.path.isdir(source_dir):
        print(f"Error: Source directory '{source_dir}' does not exist.")
        sys.exit(1)

    moved_count = 0
    mismatched_files = []

    # Read the file list and move files
    with open(file_list, "r", encoding="utf-8") as f:
        for line in f:
            basename = line.strip()  # Trim spaces and newlines

            if not basename:
                continue  # Skip empty lines

            # Extract file name and extension
            base_name, ext = os.path.splitext(basename)

            # Append "_embedded" before the extension
            embedded_file_name = f"{base_name}_embedded{ext}"
            print (embedded_file_name)
            src_file = os.path.join(source_dir, embedded_file_name)
            dest_file = os.path.join(destination_dir, embedded_file_name)

            if os.path.exists(src_file):
                shutil.move(src_file, dest_file)
                print(f"Moved: {src_file} -> {dest_file}")
                moved_count += 1
            else:
                print(f"Warning: File '{src_file}' not found.")
                mismatched_files.append(basename)

    print(f"\nFile moving process completed. Total files moved: {moved_count}")
    
    if mismatched_files:
        print("\nFiles not found:")
        for file in mismatched_files:
            print(file)

move_files("/home/dsi/orrbavly/GNN_project/testing_scripts/scripts/list_to_move.txt",  "/dsi/scratch/home/dsi/orrbavly/corona_data/embeddings",
                "/dsi/scratch/home/dsi/orrbavly/corona_data/watchlist_embedding")


Copy of ADIRP0002497_TCRB_embedded.csv
Copy of ADIRP0002519_TCRB_embedded.csv
Copy of ADIRP0002887_TCRB_embedded.csv
Copy of INCOV004-AC-5_TCRB_embedded.csv
Copy of INCOV015-AC-5_TCRB_embedded.csv
Copy of INCOV019-BL-5_TCRB_embedded.csv
Copy of INCOV026-AC-3_TCRB_embedded.csv
Copy of INCOV027-CV-3_TCRB_embedded.csv
Copy of INCOV033-BL-3_TCRB_embedded.csv
Copy of INCOV042-AC-3_TCRB_embedded.csv
Copy of INCOV042-CV-3_TCRB_embedded.csv
Copy of INCOV043-BL-3_TCRB_embedded.csv
Copy of INCOV044-AC-3_TCRB_embedded.csv
Copy of INCOV048-BL-3_TCRB_embedded.csv
Copy of INCOV054-BL-3_TCRB_embedded.csv
Copy of INCOV058-BL-3_TCRB_embedded.csv
Copy of INCOV068-BL-3_TCRB_embedded.csv
Copy of INCOV069-AC-3_TCRB_embedded.csv
Copy of INCOV077-BL-3_TCRB_embedded.csv
Copy of ADIRP0002301_TCRB_embedded.csv
Copy of ADIRP0002425_TCRB_embedded.csv
Copy of ADIRP0002512_TCRB_embedded.csv
Copy of BS-EQ-34-T1-replacement_TCRB_embedded.csv
Copy of BS-GIGI_41-replacement_TCRB_embedded.csv
Copy of BS-GIGI_89-replacem

In [1]:
import os

original_dir = "/dsi/scratch/home/dsi/orrbavly/corona_data/original_files/"
embeddings_dir = "/dsi/scratch/home/dsi/orrbavly/corona_data/embeddings/"

# Get the list of files in original_dir and embeddings_dir
original_files = os.listdir(original_dir)
embedding_files = os.listdir(embeddings_dir)

# Extract base names from embedding_files to check against original_files
embedding_base_names = [os.path.splitext(file_name)[0].replace("_embedded", "") for file_name in embedding_files]

# Check for files in original_files that are not in embedding_files
missing_files = [file for file in original_files if os.path.splitext(file)[0] not in embedding_base_names]

# Count and print the missing files
print(f"Number of files in original_dir but not in embeddings_dir: {len(missing_files)}")
print("Missing files:")
for file in missing_files:
    print(file)


Number of files in original_dir but not in embeddings_dir: 247
Missing files:
Copy of INCOV039-BL-3_TCRB.csv
Copy of INCOV052-AC-3_TCRB.csv
Copy of INCOV066-BL-3_TCRB.csv
Copy of INCOV081-AC-3_TCRB.csv
Copy of KH20-09657_TCRB.csv
Copy of KH20-09672_TCRB.csv
Copy of KH20-09682_TCRB.csv
Copy of KH20-09687_TCRB.csv
Copy of KH20-09701_TCRB.csv
Copy of KH20-09704_TCRB.csv
Copy of KH20-09705_TCRB.csv
Copy of KH20-09706_TCRB.csv
Copy of KH20-09709_TCRB.csv
Copy of KH20-09710_TCRB.csv
Copy of KH20-09712_TCRB.csv
Copy of KH20-09714_TCRB.csv
Copy of KH20-09717_TCRB.csv
Copy of KH20-09718_TCRB.csv
Copy of KH20-09721_TCRB.csv
Copy of KH20-09724_TCRB.csv
Copy of KH20-09729_TCRB.csv
Copy of KH20-09731_TCRB.csv
Copy of KH20-09752_TCRB.csv
Copy of KH20-09951_TCRB.csv
Copy of KH20-09961_TCRB.csv
Copy of KH20-09962_TCRB.csv
Copy of KH20-09964_TCRB.csv
Copy of KH20-09965_TCRB.csv
Copy of KH20-09967_TCRB.csv
Copy of KH20-09973_TCRB.csv
Copy of KH20-09974_TCRB.csv
Copy of KH20-09978_TCRB.csv
Copy of KH20-0

# Analyzing Corona data

In [2]:
# Path to the directory containing the copied files
directory_path = "/dsi/scratch/home/dsi/orrbavly/corona_data/original_files/"

# Column name to rename
old_column_name = "amino_acid"
new_column_name = "Sequences"

# Loop through all files in the directory
for file_name in os.listdir(directory_path):
    if file_name.endswith(".tsv"):  # Process only CSV files
        file_path = os.path.join(directory_path, file_name)
        
        # Load the CSV file
        df = pd.read_csv(file_path, sep='\t')
        
        # Rename the column if it exists
        if old_column_name in df.columns:
            df.rename(columns={old_column_name: new_column_name}, inplace=True)
            print(f"Renaming column in: {file_name}")
        
        # Save the updated CSV back to the same location
        df.to_csv(file_path, index=False)

print("All column names updated.")

Renaming column in: 03845000013199_TCRB.tsv
Renaming column in: 03855000011610_TCRB.tsv
Renaming column in: 03855000011616_TCRB.tsv
Renaming column in: 03855000011623_TCRB.tsv
Renaming column in: 03855000011624_TCRB.tsv
Renaming column in: 03855000011628_TCRB.tsv
Renaming column in: 03855000011629_TCRB.tsv
Renaming column in: 03855000011630_TCRB.tsv
Renaming column in: 03855000011635_TCRB.tsv
Renaming column in: 03855000011646_TCRB.tsv
Renaming column in: 03855000011647_TCRB.tsv
Renaming column in: 03855000011650_TCRB.tsv
Renaming column in: 03855000011651_TCRB.tsv
Renaming column in: 03855000011652_TCRB.tsv
Renaming column in: 03855000011656_TCRB.tsv
Renaming column in: 03855000011657_TCRB.tsv
Renaming column in: 03855000011658_TCRB.tsv
Renaming column in: 03855000011662_TCRB.tsv
Renaming column in: 03855000011664_TCRB.tsv
Renaming column in: 03855000011665_TCRB.tsv
Renaming column in: 03855000011667_TCRB.tsv
Renaming column in: 03855000011671_TCRB.tsv
Renaming column in: 038550000116

In [2]:
# Define the input directory containing TSV files
input_dir = "/dsi/scratch/home/dsi/orrbavly/corona_data/original_files/"
# Loop through all files in the input directory
for file_name in os.listdir(input_dir):
    if file_name.endswith(".tsv"):  # Process only .tsv files
        # Construct the full file path
        file_path = os.path.join(input_dir, file_name)
        
        # Load the TSV file
        df = pd.read_csv(file_path, sep="\t")
        
        # Overwrite the file as a CSV (same name, but .csv extension)
        new_file_path = os.path.join(input_dir, file_name.replace(".tsv", ".csv"))
        df.to_csv(new_file_path, index=False)
        
        # Optionally, remove the old .tsv file
        os.remove(file_path)
        print(f"Converted and replaced {file_name} with {os.path.basename(new_file_path)}")

print("All TSV files have been successfully converted to CSV.")



Converted and replaced 03845000013199_TCRB.tsv with 03845000013199_TCRB.csv
Converted and replaced 03855000011610_TCRB.tsv with 03855000011610_TCRB.csv
Converted and replaced 03855000011616_TCRB.tsv with 03855000011616_TCRB.csv
Converted and replaced 03855000011623_TCRB.tsv with 03855000011623_TCRB.csv
Converted and replaced 03855000011624_TCRB.tsv with 03855000011624_TCRB.csv
Converted and replaced 03855000011628_TCRB.tsv with 03855000011628_TCRB.csv
Converted and replaced 03855000011629_TCRB.tsv with 03855000011629_TCRB.csv
Converted and replaced 03855000011630_TCRB.tsv with 03855000011630_TCRB.csv
Converted and replaced 03855000011635_TCRB.tsv with 03855000011635_TCRB.csv
Converted and replaced 03855000011646_TCRB.tsv with 03855000011646_TCRB.csv
Converted and replaced 03855000011647_TCRB.tsv with 03855000011647_TCRB.csv
Converted and replaced 03855000011650_TCRB.tsv with 03855000011650_TCRB.csv
Converted and replaced 03855000011651_TCRB.tsv with 03855000011651_TCRB.csv
Converted an

In [3]:
import os
import pandas as pd

# Directory containing the CSV files with the combined column and combined values
directory_path = "/dsi/scratch/home/dsi/orrbavly/corona_data/original_files/"

# Loop through all CSV files in the directory
for file_name in os.listdir(directory_path):
    if file_name.endswith(".csv"):  # Process only CSV files
        file_path = os.path.join(directory_path, file_name)
        
        # Load the CSV file
        df = pd.read_csv(file_path)
        
        # Check if the combined column exists
        if "Sequences,templates" in df.columns:
            # Split the combined column into two separate columns
            df[['Sequences', 'templates']] = df['Sequences,templates'].str.split(',', expand=True)
            
            # Drop the original combined column
            df.drop(columns=["Sequences,templates"], inplace=True)
            
            # Save the fixed file back
            df.to_csv(file_path, index=False)
            print(f"Fixed and saved: {file_name}")

print("All files have been fixed.")


Fixed and saved: 03855000011616_TCRB.csv
Fixed and saved: 03855000011623_TCRB.csv
Fixed and saved: 03855000011624_TCRB.csv
Fixed and saved: 03855000011628_TCRB.csv
Fixed and saved: 03855000011629_TCRB.csv
Fixed and saved: 03855000011630_TCRB.csv
Fixed and saved: 03855000011635_TCRB.csv
Fixed and saved: 03855000011647_TCRB.csv
Fixed and saved: 03855000011650_TCRB.csv
Fixed and saved: 03855000011651_TCRB.csv
Fixed and saved: 03855000011652_TCRB.csv
Fixed and saved: 03855000011656_TCRB.csv
Fixed and saved: 03855000011657_TCRB.csv
Fixed and saved: 03855000011658_TCRB.csv
Fixed and saved: 03855000011664_TCRB.csv
Fixed and saved: 03855000011665_TCRB.csv
Fixed and saved: 03855000011667_TCRB.csv
Fixed and saved: 03855000011671_TCRB.csv
Fixed and saved: 03855000011677_TCRB.csv
Fixed and saved: 03855000011688_TCRB.csv
Fixed and saved: 03855000011696_TCRB.csv
Fixed and saved: 03855000011704_TCRB.csv
Fixed and saved: 03855000011709_TCRB.csv
Fixed and saved: 03855000011730_TCRB.csv
Fixed and saved:

In [3]:
import numpy as np
import pandas as pd
import os
import torch
from torch_cluster import radius
import time
from scipy.spatial.distance import pdist, squareform

In [4]:
METRIC = 'cosine'
PERCENTILES = [5, 15, 25, 35, 50, 70, 80, 90, 95]
OUTPUT_FILE = "/home/dsi/orrbavly/GNN_project/embeddings/ovarian_percentiles/perc_ball_cos_3.json"
EMBEDDINGS_FOLDER = "/dsi/sbm/OrrBavly/ovarian_data/embeddings/"

# Radius-Based Methods

In [5]:
def analyze_ball(file_path):
    df = pd.read_csv(file_path)
    df = pd.read_csv(file_path)
    embeddings = df.iloc[:, 1:].values.astype('float32')
    
    # Compute pairwise distances
    # Compute distances
    distances = pdist(embeddings, metric='cosine')
    if np.isnan(distances).any():
        print(f"Skipping file {file_path}: NaN values in distances.")
        return None

    # Calculate statistics
    min_dist = np.min(distances)
    max_dist = np.max(distances)
    mean_dist = np.mean(distances)
    percentiles = np.percentile(distances, [25, 50, 75, 90, 95, 99])

    print(f"min dits: {min_dist}\nmaxdist: {max_dist}\nmean dist: {mean_dist}\npercentiles: {percentiles}")

# Create json ouput file of percentiles
def run_percentiles():
    embeddings_folder = EMBEDDINGS_FOLDER
    output_file = OUTPUT_FILE
    files = os.listdir(embeddings_folder)
    all_results = {}
    i = 1
    all_statistics = []

    print(f"Started working on Metric: {METRIC}\nPercentile: {PERCENTILES}")
    for file in files:
        # Construct full file path
        file_path = os.path.join(embeddings_folder, file)
        # Check if the file is a CSV
        if file.endswith('.csv') and 'fp' not in file:
            print(f"working on file:{file}, number {i}")
            stats = analyze_ball(file_path)
            if stats is not None:  # Only include valid results
                all_statistics.append(stats)
            print(f"finished working on file:{file}, number {i}")
            i+=1
        # Aggregate statistics across all graphs
    
    aggregated_stats = {
        "avg_min_dist": np.mean([s["min_dist"] for s in all_statistics]),
        "avg_max_dist": np.mean([s["max_dist"] for s in all_statistics]),
        "avg_mean_dist": np.mean([s["mean_dist"] for s in all_statistics]),
        "avg_percentiles": np.mean([s["percentiles"] for s in all_statistics], axis=0)
    }

    print("Aggregated Statistics:")
    print(f"Average Min Distance: {aggregated_stats['avg_min_dist']}")
    print(f"Average Max Distance: {aggregated_stats['avg_max_dist']}")
    print(f"Average Mean Distance: {aggregated_stats['avg_mean_dist']}")
    print(f"Average Percentiles: {aggregated_stats['avg_percentiles']}")
    avg_percentiles = aggregated_stats["avg_percentiles"]
    r_values = avg_percentiles[[1, 2, 3, 4]]  # Median, 75th, 90th, and 95th percentiles
    print(f"Suggested Radius Values: {r_values}")


In [ ]:
run_percentiles()

Started working on Metric: cosine
Percentile: [5, 15, 25, 35, 50, 70, 80, 90, 95]
working on file:12_nd_A_B_H.csv, number 1


In [7]:
import pandas as pd
import numpy as np
import os 

def analyze_file_for_nan(file_path):
    print(f"Analyzing file: {file_path}")
    
    # Load the file
    df = pd.read_csv(file_path)
    print(f"Data shape: {df.shape}")
    
    # Check for missing or invalid values in the entire dataframe
    if df.isnull().any().any():
        print("Missing values detected in the dataframe.")
        print("Columns with NaN values:")
        print(df.isnull().sum())
    else:
        print("No missing values detected in the dataframe.")

    # Extract embeddings
    embeddings = df.iloc[:, 1:].values.astype('float32')
    
    # Check for all-zero rows
    zero_rows = np.all(embeddings == 0, axis=1)
    print(f"Number of all-zero rows: {np.sum(zero_rows)}")
    
    # Check for rows with NaN or Inf values
    nan_rows = np.isnan(embeddings).any(axis=1)
    inf_rows = np.isinf(embeddings).any(axis=1)
    print(f"Number of rows with NaN values: {np.sum(nan_rows)}")
    print(f"Number of rows with Inf values: {np.sum(inf_rows)}")
    
    # If problematic rows exist, log their indices
    if np.sum(zero_rows) > 0:
        print("Indices of all-zero rows:")
        print(np.where(zero_rows)[0])
    if np.sum(nan_rows) > 0:
        print("Indices of rows with NaN values:")
        print(np.where(nan_rows)[0])
    if np.sum(inf_rows) > 0:
        print("Indices of rows with Inf values:")
        print(np.where(inf_rows)[0])
    
    # Check for duplicate rows
    unique_embeddings = np.unique(embeddings, axis=0)
    num_duplicates = embeddings.shape[0] - unique_embeddings.shape[0]
    print(f"Number of duplicate rows: {num_duplicates}")
    
    # Analyze distances if data is valid
    if not nan_rows.any() and not inf_rows.any() and embeddings.shape[0] > 1:
        from scipy.spatial.distance import pdist
        distances = pdist(embeddings, metric='cosine')
        print(f"Min Distance: {np.min(distances)}")
        print(f"Max Distance: {np.max(distances)}")
        print(f"Mean Distance: {np.mean(distances)}")
        print(f"Percentiles: {np.percentile(distances, [25, 50, 75, 90, 95, 99])}")
    else:
        print("Data contains issues preventing distance analysis.")


folder = "/dsi/sbm/OrrBavly/ovarian_data/embeddings/"
files = os.listdir(folder)
for file in files:
    if 'fp' in file:
        file_path = os.path.join(folder, file)
        # Replace with the problematic file's path
        # file_path = "/dsi/sbm/OrrBavly/ovarian_data/embeddings/34_A_B_OC_fp.csv"
        analyze_file_for_nan(file_path)

Analyzing file: /dsi/sbm/OrrBavly/ovarian_data/embeddings/34_A_B_OC_fp.csv
Data shape: (8769, 769)
Missing values detected in the dataframe.
Columns with NaN values:
Sequences    0
0            0
1            0
2            0
3            0
            ..
763          1
764          1
765          1
766          1
767          1
Length: 769, dtype: int64
Number of all-zero rows: 0
Number of rows with NaN values: 1
Number of rows with Inf values: 0
Indices of rows with NaN values:
[8768]
Number of duplicate rows: 0
Data contains issues preventing distance analysis.
Analyzing file: /dsi/sbm/OrrBavly/ovarian_data/embeddings/31_A_B_OC_fp.csv
Data shape: (15937, 769)
No missing values detected in the dataframe.
Number of all-zero rows: 0
Number of rows with NaN values: 0
Number of rows with Inf values: 0
Number of duplicate rows: 0
Min Distance: 1.052722742311829e-06
Max Distance: 0.925080112185596
Mean Distance: 0.526754336038469
Percentiles: [0.45885421 0.52781019 0.60209841 0.64812525 0.

In [5]:
import pandas as pd
import numpy as np
from torch_cluster import radius
import torch
from scipy.spatial.distance import pdist, squareform


def analyze_with_pdist(file_path, r_values):
    """
    Analyzes graph statistics for different radius values using pdist for cosine distance.
    
    Args:
        embeddings (np.ndarray): Node embeddings (num_nodes x num_features).
        r_values (list): List of radius (r) values to analyze.
    
    Returns:
        dict: Statistics for each radius value.
    """
    df = pd.read_csv(file_path)
    embeddings = df.iloc[:, 1:].values.astype('float32')
    # Compute cosine distances using pdist
    distances = pdist(embeddings, metric='cosine')  # Pairwise cosine distances
    distance_matrix = squareform(distances)  # Convert to square matrix for easy indexing
    num_nodes = embeddings.shape[0]

    # Store results for each radius
    results = {}

    # Analyze each radius value
    for r in r_values:
        print(f"Analyzing radius: {r}")

        # Create adjacency matrix based on radius
        adj_matrix = (distance_matrix <= r).astype(int)

        # Calculate node degrees
        degrees = adj_matrix.sum(axis=1)

        # Graph statistics
        avg_degree = degrees.mean()
        max_degree = degrees.max()
        min_degree = degrees.min()
        sparsity = 1 - (degrees.sum() / (num_nodes * num_nodes))  # Fraction of zero entries in adjacency matrix

        # Save stats for this radius
        results[r] = {
            "num_nodes": num_nodes,
            "avg_degree": avg_degree,
            "max_degree": max_degree,
            "min_degree": min_degree,
            "sparsity": sparsity
        }
        print(f"  Avg Degree: {avg_degree}, Max Degree: {max_degree}, Min Degree: {min_degree}, Sparsity: {sparsity:.4f}")

    return results


def analyze_r_values(file_path, r_values):
    """
    Analyzes the impact of different radius values (r) on graph statistics.
    
    Args:
        file_path (str): Path to the CSV file containing embeddings.
        r_values (list): List of radius values to analyze.

    Returns:
        dict: Statistics for each radius value.
    """
    # Load embeddings from file
    df = pd.read_csv(file_path)
    embeddings = df.iloc[:, 1:].values.astype('float32')
    num_nodes = embeddings.shape[0]

    # Results dictionary
    results = {}

    # Analyze each radius value
    for r in r_values:
        print(f"Analyzing radius: {r}")

        # Compute ball-query-based neighbors
        edge_index = radius(torch.tensor(embeddings), torch.tensor(embeddings), r)
        
        # Calculate node degrees
        degrees = np.bincount(edge_index[0].numpy(), minlength=num_nodes)

        # Graph statistics
        avg_degree = degrees.mean()
        max_degree = degrees.max()
        min_degree = degrees.min()
        sparsity = 1 - (degrees.sum() / (num_nodes * num_nodes))  # Fraction of zero entries in adjacency matrix

        # Save stats for this radius
        results[r] = {
            "num_nodes": num_nodes,
            "avg_degree": avg_degree,
            "max_degree": max_degree,
            "min_degree": min_degree,
            "sparsity": sparsity
        }
        print(f"  Avg Degree: {avg_degree}, Max Degree: {max_degree}, Min Degree: {min_degree}, Sparsity: {sparsity:.4f}")

    return results



In [ ]:

# Example usage
file_path = "/dsi/sbm/OrrBavly/ovarian_data/embeddings/1_A_B_H.csv"
r_values = [1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0]  # Example radius values
stats = analyze_r_values(file_path, r_values)

# Print summarized results
print("\nSummary of Radius Impact:")
for r, metrics in stats.items():
    print(f"Radius {r}: {metrics}")

In [6]:
# Example usage
file_path = "/dsi/sbm/OrrBavly/ovarian_data/embeddings/1_A_B_H.csv"
r_values = [0.45, 0.5, 0.53, 0.6, 0.65, 0.68, 0.72]  # Example radius values
stats = analyze_with_pdist(file_path, r_values)

# Print summarized results
print("\nSummary of Radius Impact:")
for r, metrics in stats.items():
    print(f"Radius {r}: {metrics}")

Analyzing radius: 0.45
  Avg Degree: 3192.161280780892, Max Degree: 8908, Min Degree: 2, Sparsity: 0.7639
Analyzing radius: 0.5
  Avg Degree: 5966.772535679953, Max Degree: 10613, Min Degree: 3, Sparsity: 0.5588
Analyzing radius: 0.53
  Avg Degree: 7465.379501589884, Max Degree: 11495, Min Degree: 3, Sparsity: 0.4479
Analyzing radius: 0.6
  Avg Degree: 10476.904163277379, Max Degree: 13185, Min Degree: 43, Sparsity: 0.2253
Analyzing radius: 0.65
  Avg Degree: 12407.352288693337, Max Degree: 13494, Min Degree: 300, Sparsity: 0.0825
Analyzing radius: 0.68
  Avg Degree: 13079.618353915552, Max Degree: 13519, Min Degree: 849, Sparsity: 0.0328
Analyzing radius: 0.72
  Avg Degree: 13434.289210973897, Max Degree: 13523, Min Degree: 2933, Sparsity: 0.0066

Summary of Radius Impact:
Radius 0.45: {'num_nodes': 13523, 'avg_degree': np.float64(3192.161280780892), 'max_degree': np.int64(8908), 'min_degree': np.int64(2), 'sparsity': np.float64(0.7639457752879619)}
Radius 0.5: {'num_nodes': 13523, 'a

## pdist

In [7]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform
import torch

def analyze_embedding_file(file_path):
    """
    Analyzes the embeddings in a file to check value ranges, distance distribution,
    and the effect of normalization on cosine distances.

    Args:
        file_path (str): Path to the file containing embeddings.

    Returns:
        None
    """
    # Load embeddings
    print(f"Loading embeddings from {file_path}...")
    df = pd.read_csv(file_path)
    embeddings = df.iloc[:, 1:].values.astype('float32')  # Assuming first column is non-embedding (e.g., ID)
    print(f"Shape of embeddings: {embeddings.shape}")

    # Check embedding value range
    print("\n--- Embedding Value Range ---")
    min_val = np.min(embeddings)
    max_val = np.max(embeddings)
    mean_val = np.mean(embeddings)
    print(f"Min Value: {min_val}, Max Value: {max_val}, Mean Value: {mean_val}")

    # Compute distance distribution with SciPy (cosine distance)
    print("\n--- Distance Distribution (Original Embeddings) ---")
    distances = pdist(embeddings, metric='cosine')
    print("Percentiles (Cosine Distance):", np.percentile(distances, [25, 50, 75, 90, 95]))

    # Normalize embeddings
    print("\n--- Normalizing Embeddings ---")
    normalized_embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
    normalized_distances = pdist(normalized_embeddings, metric='cosine')
    print("Percentiles (Normalized Cosine Distance):", np.percentile(normalized_distances, [25, 50, 75, 90, 95]))

    # Analyze normalized embeddings with torch
    embeddings_torch = torch.tensor(embeddings, dtype=torch.float32)
    normalized_torch = torch.nn.functional.normalize(embeddings_torch, p=2, dim=1)

    # Compute cosine similarity and distances in torch
    similarity_matrix = torch.matmul(normalized_torch, normalized_torch.T)
    distance_matrix = 1 - similarity_matrix
    print("\n--- Distance Distribution (Torch, Normalized) ---")
    print("Percentiles (Torch Distance):", np.percentile(distance_matrix.numpy(), [25, 50, 75, 90, 95]))

    # Test ball query with torch_cluster.radius
    from torch_cluster import radius

    print("\n--- Ball Query Results (Torch) ---")
    r_values = [0.1, 0.2, 0.3, 0.4, 0.5]  # Adjust as needed
    num_nodes = embeddings.shape[0]
    for r in r_values:
        edge_index = radius(normalized_torch, normalized_torch, r)
        degrees = np.bincount(edge_index[0].numpy(), minlength=num_nodes)
        avg_degree = degrees.mean()
        print(f"Radius {r}: Avg Degree = {avg_degree}, Max Degree = {degrees.max()}, Min Degree = {degrees.min()}")

if __name__ == "__main__":
    # Replace with your file path
    file_path = "/dsi/sbm/OrrBavly/ovarian_data/embeddings/1_A_B_H.csv"
    analyze_embedding_file(file_path)


Loading embeddings from /dsi/sbm/OrrBavly/ovarian_data/embeddings/1_A_B_H.csv...
Shape of embeddings: (13523, 768)

--- Embedding Value Range ---
Min Value: -4.329875946044922, Max Value: 2.9154725074768066, Mean Value: -0.0006522214389406145

--- Distance Distribution (Original Embeddings) ---
Percentiles (Cosine Distance): [0.45366544 0.51525968 0.59233865 0.64246004 0.66729372]

--- Normalizing Embeddings ---
Percentiles (Normalized Cosine Distance): [0.45366544 0.51525969 0.59233865 0.64246004 0.66729372]

--- Distance Distribution (Torch, Normalized) ---
Percentiles (Torch Distance): [0.45365155 0.51524979 0.59233284 0.64245695 0.66729134]

--- Ball Query Results (Torch) ---
Radius 0.1: Avg Degree = 1.000147896176884, Max Degree = 2, Min Degree = 1
Radius 0.2: Avg Degree = 1.0023663388301411, Max Degree = 2, Min Degree = 1
Radius 0.3: Avg Degree = 1.0735043999112623, Max Degree = 10, Min Degree = 1
Radius 0.4: Avg Degree = 1.6759594764475338, Max Degree = 32, Min Degree = 1
Radius

## torch_cluster radius

In [1]:
import pandas as pd
import numpy as np
import torch
from torch_cluster import radius
import os
import scipy.sparse as sp

def normalize_embeddings(embeddings):
    """
    Normalize embeddings to unit vectors (float64 for precision).
    """
    embeddings = embeddings.astype(np.float64)
    return embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

def create_sparse_adjacency_matrix(embeddings, r):
    """
    Create a sparse adjacency matrix using ball queries based on cosine distance.
    
    Args:
        embeddings (np.ndarray): Normalized embeddings (num_nodes x num_features).
        r (float): Radius for ball query.

    Returns:
        scipy.sparse.csr_matrix: Sparse adjacency matrix.
    """
    num_nodes = embeddings.shape[0]

    # Convert to float64 tensor for precision
    embeddings_torch = torch.tensor(embeddings, dtype=torch.float64)

    # Perform ball query (returns edge indices)
    edge_index = radius(embeddings_torch, embeddings_torch, r)

    # Convert edge indices to NumPy
    row_indices = edge_index[0].numpy()
    col_indices = edge_index[1].numpy()
    values = np.ones_like(row_indices, dtype=np.float64)  # All edges = 1

    # Create sparse adjacency matrix
    adj_matrix = sp.csr_matrix((values, (row_indices, col_indices)), shape=(num_nodes, num_nodes))
    
    return adj_matrix

def analyze_files(embedding_folder, r_values, batch_size=5000):
    """
    Process multiple embedding files, generate sparse adjacency matrices, and compute graph statistics.

    Args:
        embedding_folder (str): Folder containing CSV embedding files.
        r_values (list): List of radius values to test.
        output_folder (str): Folder to save adjacency matrices and results.
        batch_size (int): Optional batch size for processing large embeddings.

    Returns:
        None
    """
    files = [f for f in os.listdir(embedding_folder) if f.endswith('.csv')]
    files_to_analyze = ["12_nd_A_B_H.csv", "23_A_B_H.csv", "22_nd_A_B_H.csv", "11_nd_A_B_OC.csv",
                         "27_A_B_H.csv", "6_A_B_H.csv", "19_A_B_OC.csv"]
    results = []
    counter = 0 
    for file in files:
        if "fp" not in file and os.path.basename(file) in files_to_analyze:
            print(f"Processing file: {file}")
            file_path = os.path.join(embedding_folder, file)
            # Load and normalize embeddings
            df = pd.read_csv(file_path)
            embeddings = df.iloc[:, 1:].values.astype(np.float64)
            normalized_embeddings = normalize_embeddings(embeddings)
            num_nodes = normalized_embeddings.shape[0]
            for r in r_values:
                print(f"  Analyzing radius: {r}")

                # Create sparse adjacency matrix
                adj_matrix = create_sparse_adjacency_matrix(normalized_embeddings, r)

                # Calculate graph statistics
                degrees = np.array(adj_matrix.sum(axis=1)).flatten()
                avg_degree = np.mean(degrees)
                max_degree = np.max(degrees)
                min_degree = np.min(degrees)
                sparsity = 1 - (degrees.sum() / (num_nodes * num_nodes))
                results.append({
                    "file": file,
                    "radius": r,
                    "num_nodes": num_nodes,
                    "avg_degree": avg_degree,
                    "max_degree": max_degree,
                    "min_degree": min_degree,
                    "sparsity": sparsity
                })

                print(f"    Avg Degree: {avg_degree}, Max Degree: {max_degree}, Min Degree: {min_degree}, Sparsity: {sparsity:.4f}")


if __name__ == "__main__":
    EMBEDDING_FOLDER = "/dsi/sbm/OrrBavly/ovarian_data/embeddings/" 
    R_VALUES = [0.45, 0.5, 0.53, 0.6, 0.65, 0.68, 0.72]  # Radius values to test

    analyze_files(EMBEDDING_FOLDER, R_VALUES)


Processing file: 12_nd_A_B_H.csv
  Analyzing radius: 0.45
    Avg Degree: 3.126079776422764, Max Degree: 32.0, Min Degree: 1.0, Sparsity: 0.9998
  Analyzing radius: 0.5
    Avg Degree: 4.8993267276422765, Max Degree: 32.0, Min Degree: 1.0, Sparsity: 0.9997
  Analyzing radius: 0.53
    Avg Degree: 6.106961382113822, Max Degree: 32.0, Min Degree: 1.0, Sparsity: 0.9996
  Analyzing radius: 0.6
    Avg Degree: 10.357533028455284, Max Degree: 32.0, Min Degree: 1.0, Sparsity: 0.9993
  Analyzing radius: 0.65
    Avg Degree: 16.788363821138212, Max Degree: 32.0, Min Degree: 1.0, Sparsity: 0.9989
  Analyzing radius: 0.68
    Avg Degree: 21.848259654471544, Max Degree: 32.0, Min Degree: 1.0, Sparsity: 0.9986
  Analyzing radius: 0.72
    Avg Degree: 27.787093495934958, Max Degree: 32.0, Min Degree: 1.0, Sparsity: 0.9982
Processing file: 23_A_B_H.csv
  Analyzing radius: 0.45
    Avg Degree: 6.4153426606592365, Max Degree: 32.0, Min Degree: 1.0, Sparsity: 0.9996
  Analyzing radius: 0.5
    Avg Degre

## Radius with cdist